次のノートブックはGitHub Copilot Chatによって自動生成されており、初期設定のみを目的としています


# プロンプトエンジニアリング入門
プロンプトエンジニアリングとは、自然言語処理タスクのためにプロンプトを設計し、最適化するプロセスです。適切なプロンプトの選定やパラメータの調整、そしてそのパフォーマンスの評価が含まれます。プロンプトエンジニアリングは、NLPモデルで高い精度と効率を実現するために欠かせません。このセクションでは、OpenAIモデルを使ってプロンプトエンジニアリングの基本を学んでいきます。


### 演習 1: トークナイゼーション
OpenAIが提供する高速トークナイザー「tiktoken」を使ってトークナイゼーションを体験しましょう。
さらに詳しい例は [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) をご覧ください。


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### 演習2: OpenAI APIキーの設定を確認する

下記のコードを実行して、OpenAIエンドポイントが正しく設定されているか確認しましょう。このコードはシンプルなプロンプトを試し、応答が正しいかを検証します。入力として `oh say can you see` を与えると、`by the dawn's early light..` のような続きが返ってくるはずです。


In [ ]:
# The OpenAI SDK was updated on Nov 8, 2023 with new guidance for migration
# See: https://github.com/openai/openai-python/discussions/742

## Updated
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

## Updated
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]       
    response = client.chat.completions.create(   
        model=deployment,                                         
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt)
print(response)

### 演習3: 捏造

存在しないトピックや、事前学習データセットの範囲外（より最近の話題など）でLLMにプロンプトを与えた場合、どのような回答が返ってくるかを試してみましょう。別のプロンプトや別のモデルを使うと、どのように応答が変化するかも観察してみてください。


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt)
print(response)

### 演習4: 指示ベース
"text" 変数に主要な内容を設定し、"prompt" 変数にその内容に関連した指示を記述します。

ここでは、モデルに対して、テキストを小学2年生向けに要約するように求めます。


In [ ]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt)
print(response)

### 演習5: 複雑なプロンプト
システム、ユーザー、アシスタントのメッセージを含むリクエストを試してみましょう  
システムはアシスタントのコンテキストを設定します  
ユーザーとアシスタントのメッセージは複数ターンの会話コンテキストを提供します

システムコンテキストでアシスタントの性格が「皮肉屋」に設定されている点に注目してください。  
別の性格コンテキストを使ってみたり、異なる一連の入力/出力メッセージを試してみても良いでしょう


In [ ]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

### 演習：直感を探求しよう
上記の例は、新しいプロンプト（シンプルなもの、複雑なもの、指示型など）を作成する際に使えるパターンを示しています。他にも、例やヒントなど、これまでに話したさまざまなアイデアを探求するための演習を自分で作ってみましょう。



---

**免責事項**:  
本書類は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性には努めておりますが、自動翻訳には誤りや不正確な表現が含まれる場合があります。原文（元の言語の文書）が正式な情報源と見なされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じたいかなる誤解や誤訳についても、当方は一切の責任を負いかねます。
